# TP1
## Grupo 15

Carlos Eduardo Da Silva Machado A96936
Gonçalo Manuel Maia de Sousa A97485

## Problema 1

Pretende-se construir um horário semanal para o plano de reuniões de projeto de uma “StartUp” de acordo com as seguintes condições:
    1. Cada reunião ocupa uma sala (enumeradas $$1...S\,$$) durante um “slot”  $$1..T\,$$ $$(\text{hora},\text{dia})$$.  
    2.  Cada reunião tem associado um projeto (enumerados $$1..P$$) e um conjunto de participantes. Os diferentes colaboradores são enumerados $$1..C$$.
    3. Cada projeto tem associado um conjunto de colaboradores, dos quais um  é o líder. Cada projeto realiza um dado número de reuniões semanais. 
    4. O líder do projeto participa em todas as reuniões do seu projeto; os restantes colaboradores podem ou não participar consoante a sua disponibilidade, num mínimo (“quorum”) de  $$50\%$$ do total de colaboradores do projeto. 

In [ ]:
from ortools.linear_solver import pywraplp
horario = pywraplp.Solver.CreateSolver('SCIP')

In [ ]:
# Criação da matriz de alocação
X = {}
for p in range(P):
    for c in range(C):
        for s in range(S):
            for t in range(T):
                X[p,c,s,t] = horario.BoolVar(f'X[{p}][{c}][{s}][{t}]')

In [ ]:
# construção da matriz 2x2 da disponibilidade de cada participante
d = {}
for c in C:
    for t in T:
        d[c,t] = horario.BoolVar(f'd[{c}][{t}]')

In [ ]:
# O lider participa em todas as reuniões do projeto
for p in range(P):
    for c in range(C):
        for s in range(S):
            for t in range(T):
                horario.Add(X[p,l,s,t] >= X[p,c,s,t])


In [ ]:
# O projeto tem mínimo (“quorum”) de 50% do total de colaboradores do projeto. 
for p in range(P):
    for s in range(S):
        for t in range(T):
            horario.Add(sum([ X[p,c,s,t] for c in range(C)]) >= sum([ col[c,p] for c in range(C) ])/2)
                

In [ ]:
# Verificar se o colaborador tem disponibilidade
for c in range(C):
    for t in range(T):
        if d[c,t] == 0:
            horario.Add(sum([ X[p,c,s,t] for p in range(P) for s in range(S)] == 0))

In [ ]:
# Número Mínimo de reuniões semanais
for p in range(P):
    horario.Add(sum([ X[p,c,s,t] for c in range(C) for s in range(S) for t in range(T)]) >= m[p])

In [ ]:
# Colaboradores não podem participar em reuniões de projetos que não estão associados
for p in range(P):
    for c in range(C):
        for s in range(S):
            for t in range(T):
                horario.Add(X[p,c,s,t] <= col[p,c])

In [ ]:
# Um colaborador não pode participar em duas reuniões ao mesmo tempo
for c in range(C):
    for t in range(T):
        horario.Add(sum([ X[p,c,s,t] for p in range(P) for s in range(S)]) <= 1)

In [ ]:
# S = Int
# C = Int
# T = (hora, dia)
# P = ([conjunto de colaboradores], lider, número de reuniões semanais)

# Maximizar o número de reuniões efetivamente realizadas
# Minimizar o número médio de reuniões por participante.